# PCA of NOAA-GridSat-B1

In this notebook, we will perform Principle Component Analysis on the NOAA-GridSat-B1 data in 1998 \~ 2018 (21 years). From the previous notebook we found that data quality of NOAA-GridSat-B1 had significant improvement after 2013 (less peaks on min/max values, i.e., less outliers), we will also perform PCA on 2013~2018 data to see if there are differences.

